<a href="https://colab.research.google.com/github/wdon021/From_Colab/blob/master/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
!pip install fastai

In [0]:
from fastai.tabular import *
from sklearn import model_selection as ms
from sklearn.utils import resample

In [0]:
PATH = "/content/gdrive/My Drive/Fastai/"

In [0]:
import pandas as pd

In [0]:
h1b_raw = pd.read_csv(f'{PATH}H-1B_Disclosure_RAW_Data_FY18.csv', delimiter=',', engine='python')

In [0]:
h1b_1 = h1b_raw.drop(['CASE_NUMBER', 'CASE_SUBMITTED', 'DECISION_DATE', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE'], axis=1)

In [0]:
procs = [FillMissing, Categorify, Normalize]

In [0]:
cat_vars = ['VISA_CLASS', 'EMPLOYER_NAME', 'EMPLOYER_BUSINESS_DBA',
       'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE',
       'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE',
       'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_REPRESENTING_EMPLOYER',
       'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE',
       'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMP', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
       'PUBLIC_DISCLOSURE_LOCATION', 'WORKSITE_CITY', 'WORKSITE_COUNTY',
       'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'ORIGINAL_CERT_DATE']
cont_vars = ['TOTAL_WORKERS']

In [0]:
for col in cat_vars:
    h1b_1[col] = h1b_1[col].astype('category')

In [0]:
emb = {'EMPLOYER_NAME':50, 'EMPLOYER_BUSINESS_DBA':50, 'EMPLOYER_ADDRESS':50, 'EMPLOYER_CITY':50, 'EMPLOYER_STATE':50, 'EMPLOYER_POSTAL_CODE':50, 'EMPLOYER_PHONE':50, 'AGENT_ATTORNEY_NAME':50, 'AGENT_ATTORNEY_CITY':50, 
       'JOB_TITLE':50, 'PREVAILING_WAGE':50,
       'WORKSITE_CITY':50,'PW_SOURCE_OTHER':50,
       'WAGE_RATE_OF_PAY_FROM':50, 'WAGE_RATE_OF_PAY_TO':50, 'WORKSITE_POSTAL_CODE':50, 'NAICS_CODE':50, 'WORKSITE_COUNTY':50}

In [0]:
y = h1b_1.CASE_STATUS
X = h1b_1.drop('CASE_STATUS', axis=1)

In [45]:
X_train, X_valid, y_train, y_valid = ms.train_test_split(X, y, test_size=0.3, random_state=27)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((458052, 46), (458052,), (196308, 46), (196308,))

In [0]:
dep_var = 'CASE_STATUS'
df = h1b_1[cat_vars+cont_vars+[dep_var]].copy()

In [0]:
valid_idx = range(41395)

In [0]:
data = (TabularList.from_df(df, path=PATH, cat_names=cat_vars, cont_names=cont_vars, procs=procs).split_by_idx(valid_idx).label_from_df(cols=dep_var).add_test(TabularList.from_df(X_valid, path=PATH, cat_names=cat_vars, cont_names=cont_vars)).databunch())

In [0]:
learn = tabular_learner(data, layers=[500, 100], ps=[ 0.3, 0.1], emb_szs=emb, emb_drop=0.05, metrics=accuracy)

In [40]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(5, 4)
    (1): Embedding(68396, 50)
    (2): Embedding(10795, 50)
    (3): Embedding(61560, 50)
    (4): Embedding(4920, 50)
    (5): Embedding(56, 50)
    (6): Embedding(10577, 50)
    (7): Embedding(6, 4)
    (8): Embedding(326, 41)
    (9): Embedding(67929, 50)
    (10): Embedding(1073, 80)
    (11): Embedding(3, 3)
    (12): Embedding(6672, 50)
    (13): Embedding(1022, 50)
    (14): Embedding(56, 15)
    (15): Embedding(99178, 50)
    (16): Embedding(741, 65)
    (17): Embedding(897, 72)
    (18): Embedding(2022, 50)
    (19): Embedding(49, 14)
    (20): Embedding(24, 9)
    (21): Embedding(27, 10)
    (22): Embedding(9, 5)
    (23): Embedding(26, 10)
    (24): Embedding(22, 9)
    (25): Embedding(3, 3)
    (26): Embedding(22974, 50)
    (27): Embedding(6, 4)
    (28): Embedding(5, 4)
    (29): Embedding(6, 4)
    (30): Embedding(18, 8)
    (31): Embedding(1874, 50)
    (32): Embedding(51705, 50)
    (33): Embedding(15290, 5

In [39]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
learn.model = learn.model.cuda()

In [21]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.130632,0.635802,0.857954,05:41


In [0]:
embs = list()

In [0]:
for param in learn.model.embeds.parameters():
  embs.append(param)

In [52]:
h1b_1[cat_vars[0]].value_counts()

H-1B               639519
E-3 Australian      12566
H-1B1 Singapore      1267
H-1B1 Chile          1008
Name: VISA_CLASS, dtype: int64

In [50]:
h1b_1.shape

(654360, 47)

In [0]:
# self.classes = ds.classes = OrderedDict({n:np.concatenate([['#na#'],c.cat.categories.values])
embedding_names = ['UNKNOWN'] + list(h1b_1['VISA_CLASS'].cat.categories)

In [49]:
embedding_names

['UNKNOWN', 'E-3 Australian', 'H-1B', 'H-1B1 Chile', 'H-1B1 Singapore']

In [35]:
embs

[Parameter containing:
 tensor([[ 0.0093,  0.0015, -0.0139,  0.0061],
         [ 0.1819, -0.0619,  0.2090, -0.1780],
         [-0.0186,  0.0604, -0.0300,  0.0951],
         [ 0.0182,  0.0148,  0.0272,  0.0214],
         [-0.0033,  0.0158, -0.0150,  0.0333]], device='cuda:0',
        requires_grad=True), Parameter containing:
 tensor([[-0.0483, -0.0115,  0.0074,  ...,  0.0175, -0.0139,  0.0063],
         [ 0.0057, -0.0139,  0.0059,  ...,  0.0095,  0.0089, -0.0067],
         [ 0.0088, -0.0111, -0.0071,  ..., -0.0189, -0.0023,  0.0010],
         ...,
         [-0.0244,  0.0046,  0.0072,  ...,  0.0128,  0.0135,  0.0048],
         [ 0.0236, -0.0202,  0.0042,  ..., -0.0007, -0.0015, -0.0095],
         [ 0.0269, -0.0215, -0.0191,  ..., -0.0443, -0.0384, -0.0025]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([[-0.0112, -0.0401,  0.0509,  ..., -0.0181,  0.0176,  0.0326],
         [ 0.0065, -0.0042, -0.0045,  ...,  0.0076, -0.0141,  0.0016],
         [-0.0007, -0.

In [0]:
unique_row= list(source1.index.values)

In [0]:
unique_row

In [0]:
def tokenize_corpus(corpus):
  tokens = [x.split() for x in corpus]
  return tokens

In [0]:
tokenized_corpus = tokenize_corpus(unique_row)

In [0]:
tokenized_corpus

[['OFLC', 'ONLINE', 'DATA', 'CENTER'],
 ['RADFORD', 'GLOBAL', 'TECHNOLOGY', 'SURVEY'],
 ['ONLINE', 'WAGE', 'LIBRARY'],
 ['OFLC', '(ONLINE', 'DATA', 'CENTER)'],
 ['OFLC', 'ONLINE', 'DATA', "CENTER'"],
 ['US', 'MBD', 'MERCER', 'BENCHMARK', 'DATABASE', 'IT', 'SURVEY'],
 ['DIETRICH', 'FALL', '2017', 'ENGINEERING', 'SALARY', 'SURVEY'],
 ['RADFORD', 'GLOBAL', 'TECHNOLOGY', 'SURVEY', 'CUSTOM', 'REPORT'],
 ['WILLIS',
  'TOWERS',
  'WATSON:',
  'PROFESSIONAL',
  '(TECHNICAL',
  'AND',
  'OPERATIONS)',
  'COMPENSATION',
  'SUR'],
 ['COLLECTIVE', 'BARGAINING', 'AGREEMENT'],
 ['AAMC', 'SURVEY', 'OF', 'RESIDENT/FELLOW', 'STIPENDS', 'AND', 'BENEFITS'],
 ['OFLC', 'ONLINE', 'DATACENTER'],
 ['TOWERS',
  'WATSON',
  'PROFESSIONAL',
  '(INFORMATION',
  'TECHNOLOGY)',
  'COMPENSATION',
  'SURVEY'],
 ['MERCER', 'BENCHMARK', 'DATABASE', 'SURVEY'],
 ['TOWERS',
  'WATSON',
  'PROFESSIONAL',
  '(TECHNICAL',
  'AND',
  'OPERATIONS)',
  'COMPENSATION',
  'SURVEY'],
 ['TOWERS',
  'WATSON:',
  '2017',
  'PROFESSIO

In [0]:
vocabulary = []

In [0]:
for sentence in tokenized_corpus:
  for token in sentence:
    if token not in vocabulary:
      vocabulary.append(token)

In [0]:
word2idx = {w:idx for (idx,w) in enumerate(vocabulary)}
idx2word 

['OFLC',
 'ONLINE',
 'DATA',
 'CENTER',
 'RADFORD',
 'GLOBAL',
 'TECHNOLOGY',
 'SURVEY',
 'WAGE',
 'LIBRARY',
 '(ONLINE',
 'CENTER)',
 "CENTER'",
 'US',
 'MBD',
 'MERCER',
 'BENCHMARK',
 'DATABASE',
 'IT',
 'DIETRICH',
 'FALL',
 '2017',
 'ENGINEERING',
 'SALARY',
 'CUSTOM',
 'REPORT',
 'WILLIS',
 'TOWERS',
 'WATSON:',
 'PROFESSIONAL',
 '(TECHNICAL',
 'AND',
 'OPERATIONS)',
 'COMPENSATION',
 'SUR',
 'COLLECTIVE',
 'BARGAINING',
 'AGREEMENT',
 'AAMC',
 'OF',
 'RESIDENT/FELLOW',
 'STIPENDS',
 'BENEFITS',
 'DATACENTER',
 'WATSON',
 '(INFORMATION',
 'TECHNOLOGY)',
 '(ADMINISTRATIVE',
 'SALES)',
 'SURV',
 'ACWIA',
 'WWW.FLCDATACENTER.COM',
 '&',
 'PROF.',
 'COMP.',
 'INFORMATION',
 'AIA',
 'CSR',
 'GENERAL',
 'INDUS',
 "PROF'L",
 '(TECH',
 'COMP',
 'RESIDENT',
 'FELLOW',
 'PREVAILING',
 'DETERMINED',
 'BY',
 'CBA',
 'SPRING',
 '2018',
 'CENTER-ACWIA',
 'FLC',
 'SURVE',
 'SALES',
 'COMPSOURCE',
 'COMPENSA',
 'WILLISTOWERS',
 'INDUSTRY',
 'OPER',
 'THE',
 'GROUP:',
 'CENTRE',
 '-',
 'SUPERVISO